In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

<h1>Table of Contents</h1>
<ul>
  <li><a href="#function-univ_plot">Function: <code>univ_plot</code></a></li>
  <li><a href="#function-univ_plotly">Function: <code>univ_plotly</code></a></li>
  <li><a href="#function-biv_plot">Function: <code>biv_plot</code></a></li>
</ul>/ul>
</ul>


# <a id="function-univ_plot"></a>Function: `univ_plot`

This function performs univariate analysis on a given DataFrame, plotting either categorical or continuous variables based on the parameters provided. 

## <a id="parameters"></a>Parameters

### <a id="df"></a>`df`
- **Type**: `pd.DataFrame`
  - The DataFrame containing the data to be analyzed and plotted.

### <a id="threshold"></a>`threshold`
- **Type**: `int`, default=15
  - The threshold for determining whether a variable is categorical or continuous.
  - Variables with unique values less than or equal to this threshold are considered categorical.
  - Variables with unique values greater than this threshold are considered continuous.

### <a id="categorical"></a>`categorical`
- **Type**: `bool`, default=True
  - If `True`, the function plots categorical variables using count plots.
  - If `False`, the function plots continuous variables using violin plots.

### <a id="log"></a>`log`
- **Type**: `bool`, default=False
  - If `True`, the function applies `np.log()` to continuous variables before plotting them.
  - If `False`, continuous variables are plotted as they are.


### Function code:

In [51]:
def univ_plot(df, threshold=15, categorical=True, log=False):
    import random
    import numpy as np
    
    # Determine categorical and continuous columns
    categorical_columns = [col for col in df.columns if df[col].nunique() <= threshold]
    continuous_columns = [col for col in df.columns if df[col].nunique() > threshold]
    
    if categorical:
        # Plot categorical variables
        num_plots = len(categorical_columns)
        if num_plots == 0:
            print("No categorical variables to plot.")
            return
        fig, axes = plt.subplots(nrows=(num_plots // 2) + (num_plots % 2), ncols=2, figsize=(20, 5 * ((num_plots // 2) + (num_plots % 2))))
        axes = axes.flatten()
        
        for ax, col in zip(axes, categorical_columns):
            plot = sns.countplot(x=col, data=df, hue=col, legend=False, ax=ax)
            ax.set_title(f'Count of {col}', fontsize=20)
            ax.set_xlabel(col, fontsize=18)
            ax.set_ylabel('Count', fontsize=18)
            ax.tick_params(axis='x', rotation=45, labelsize=16)
            ax.tick_params(axis='y', labelsize=16)
            
            # Annotate each bar with the count if the variable has 10 or fewer categories
            if df[col].nunique() <= 10:
                for p in plot.patches:
                    plot.annotate(format(p.get_height(), '.0f'), 
                                  (p.get_x() + p.get_width() / 2., p.get_height()), 
                                  ha = 'center', va = 'center', 
                                  xytext = (0, 9), 
                                  textcoords = 'offset points',
                                  fontsize=14)
        
        # Hide any remaining axes if the number of plots is odd
        for i in range(num_plots, len(axes)):
            fig.delaxes(axes[i])
        
        plt.tight_layout()
        plt.show()
    else:
        # Define a list of colors
        colors = [
            '#1f77b4', '#ff7f0e', '#2ca02c', 
            '#d62728', '#9467bd', '#8c564b', 
            '#e377c2', '#7f7f7f', '#bcbd22', 
            '#17becf'
        ]
        
        # Plot continuous variables
        num_plots = len(continuous_columns)
        if num_plots == 0:
            print("No continuous variables to plot.")
            return
        fig, axes = plt.subplots(nrows=(num_plots // 2) + (num_plots % 2), ncols=2, figsize=(20, 5 * ((num_plots // 2) + (num_plots % 2))))
        axes = axes.flatten()
        
        for ax, col in zip(axes, continuous_columns):
            data = np.log(df[col]) if log else df[col]
            color = random.choice(colors)
            sns.violinplot(y=data, ax=ax, color=color)
            ax.set_title(f'Violin Plot of {col}', fontsize=20)
            ax.set_ylabel(col, fontsize=18)
            ax.tick_params(axis='y', labelsize=16)
        
        # Hide any remaining axes if the number of plots is odd
        for i in range(num_plots, len(axes)):
            fig.delaxes(axes[i])
        
        plt.tight_layout()
        plt.show()

# <a id="function-univ_plotly"></a>Function: `univ_plotly`

This function performs univariate analysis on a given DataFrame with Plotly, plotting either categorical or continuous variables based on the parameters provided. 

This function is better for plotting categorical variables. Continuous variables tend to be much slower and therefore you should achieve a better performance with univ_plot.

## <a id="parameters"></a>Parameters

### <a id="df"></a>`df`
- **Type**: `pd.DataFrame`
  - The DataFrame containing the data to be analyzed and plotted.

### <a id="threshold"></a>`threshold`
- **Type**: `int`, default=15
  - The threshold for determining whether a variable is categorical or continuous.
  - Variables with unique values less than or equal to this threshold are considered categorical.
  - Variables with unique values greater than this threshold are considered continuous.

### <a id="categorical"></a>`categorical`
- **Type**: `bool`, default=True
  - If `True`, the function plots categorical variables using count plots.
  - If `False`, the function plots continuous variables using violin plots.

### <a id="log"></a>`log`
- **Type**: `bool`, default=False
  - If `True`, the function applies `np.log()` to continuous variables before plotting them.
  - If `False`, continuous variables are plotted as they are.


### Function code:

In [55]:
def univ_plotly(df, threshold=15, categorical=True, log=False):

    ### Importing the packages
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import numpy as np
    
    # Determine categorical and continuous columns
    categorical_columns = [col for col in df.columns if df[col].nunique() <= threshold]
    continuous_columns = [col for col in df.columns if df[col].nunique() > threshold]
    
    if categorical:
        # Plot categorical variables
        num_plots = len(categorical_columns)
        if num_plots == 0:
            print("No categorical variables to plot.")
            return
        
        # Create subplots with 2 columns
        fig = make_subplots(rows=(num_plots // 2) + (num_plots % 2), cols=2, subplot_titles=categorical_columns)
        
        for i, col in enumerate(categorical_columns):
            row = (i // 2) + 1
            col_num = (i % 2) + 1
            fig.add_trace(
                go.Bar(x=df[col].value_counts().index, y=df[col].value_counts().values, name=col),
                row=row, col=col_num
            )
            fig.update_xaxes(title_text=col, row=row, col=col_num)
            fig.update_yaxes(title_text='Count', row=row, col=col_num)

        fig.update_layout(height=400 * ((num_plots // 2) + (num_plots % 2)), width=1200, title_text="Categorical Variable Distribution")
        fig.show()
    else:
        # Plot continuous variables
        num_plots = len(continuous_columns)
        if num_plots == 0:
            print("No continuous variables to plot.")
            return
        
        # Create subplots with 2 columns
        fig = make_subplots(rows=(num_plots // 2) + (num_plots % 2), cols=2, subplot_titles=continuous_columns)
        
        for i, col in enumerate(continuous_columns):
            row = (i // 2) + 1
            col_num = (i % 2) + 1
            data = np.log(df[col]) if log else df[col]
            fig.add_trace(
                go.Box(y=data, name=col),
                row=row, col=col_num
            )
            fig.update_yaxes(title_text=col, row=row, col=col_num)
        
        fig.update_layout(height=400 * ((num_plots // 2) + (num_plots % 2)), width=1200, title_text="Continuous Variable Distribution")
        fig.show()

# <a id="function-biv_plot"></a>Function: `biv_plot`

This function performs bivariate analysis on a given DataFrame, plotting either categorical or continuous variables based on the parameters provided.

## <a id="parameters"></a>Parameters

### <a id="df"></a>`df`
- **Type**: `pd.DataFrame`
  - The DataFrame containing the data to be analyzed and plotted.

### <a id="x"></a>`x`
- **Type**: `str`
  - The name of the column in the DataFrame to be used as the x variable.

### <a id="y"></a>`y`
- **Type**: `str`
  - The name of the column in the DataFrame to be used as the y variable.

### <a id="threshold"></a>`threshold`
- **Type**: `int`, default=15
  - The threshold for determining whether a variable is categorical or continuous.
  - Variables with unique values less than or equal to this threshold are considered categorical.
  - Variables with unique values greater than this threshold are considered continuous.

### <a id="sample_size"></a>`sample_size`
- **Type**: `int`, default=500
  - The sample size for plotting if the dataset is large.
  - If the number of rows in the DataFrame exceeds this sample size, a random sample of this size is used for plotting continuous variables.

## <a id="returns"></a>Returns

This function does not return any values. It generates and displays plots based on the bivariate analysis of the specified variables.

## <a id="examples"></a>Examples

### Continuous vs. Continuous
For two continuous variables, the function generates a scatter plot with a regression line:
```python
biv_plot(df, x='variable1', y='variable2')


### Function code:

In [59]:
def biv_plot(df, x, y, threshold=15, sample_size=500):
    from statsmodels.graphics.mosaicplot import mosaic
    # Determine if the variables are categorical or continuous
    x_is_categorical = df[x].nunique() <= threshold
    y_is_categorical = df[y].nunique() <= threshold

    plt.figure(figsize=(10, 6))

    if not x_is_categorical and not y_is_categorical:
        # Scenario 1: Continuous x Continuous
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # Sample for scatter plot if dataset is large
        if len(df) > sample_size:
            df_sampled = df.sample(sample_size)
        else:
            df_sampled = df
        
        sns.regplot(x=x, y=y, data=df_sampled, ax=ax, scatter_kws={'s': 50, 'alpha': 0.5}, line_kws={'color': 'red'}) # Scatter Plot with Regression Line
        ax.set_title(f'Scatter Plot with Regression Line: {x} vs {y}')
        ax.set_xlabel(x)
        ax.set_ylabel(y)
        
        plt.tight_layout()
        plt.show()

    elif x_is_categorical and not y_is_categorical:
        # Scenario 2: Continuous x Categorical
        fig, axs = plt.subplots(2, 2, figsize=(16, 12))

        sns.boxplot(x=x, y=y, data=df, ax=axs[0, 0], hue=x) # Box Plot
        axs[0, 0].set_title('Box Plot')
        axs[0, 0].tick_params(axis='x', rotation=45)

        sns.violinplot(x=x, y=y, data=df, ax=axs[0, 1], hue=x) # Violin Plot
        axs[0, 1].set_title('Violin Plot')
        axs[0, 1].tick_params(axis='x', rotation=45)

        sns.stripplot(x=x, y=y, data=df, jitter=True, ax=axs[1, 0], hue=x) # Strip Plot
        axs[1, 0].set_title('Strip Plot')
        axs[1, 0].tick_params(axis='x', rotation=45)

        sns.barplot(x=x, y=y, data=df, ax=axs[1, 1], hue=x) # Bar Plot
        axs[1, 1].set_title('Bar Plot')
        axs[1, 1].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.show()

    elif not x_is_categorical and y_is_categorical:
        # Scenario 2: Categorical x Continuous (swapped)
        fig, axs = plt.subplots(2, 2, figsize=(16, 12))

        sns.boxplot(x=y, y=x, data=df, ax=axs[0, 0], hue=y) # Box Plot
        axs[0, 0].set_title('Box Plot')
        axs[0, 0].tick_params(axis='x', rotation=45)

        sns.violinplot(x=y, y=x, data=df, ax=axs[0, 1], hue=y) # Violin Plot
        axs[0, 1].set_title('Violin Plot')
        axs[0, 1].tick_params(axis='x', rotation=45)

        sns.stripplot(x=y, y=x, data=df, jitter=True, ax=axs[1, 0], hue=y) # Strip Plot
        axs[1, 0].set_title('Strip Plot')
        axs[1, 0].tick_params(axis='x', rotation=45)

        sns.barplot(x=y, y=x, data=df, ax=axs[1, 1], hue=y) # Bar Plot
        axs[1, 1].set_title('Bar Plot')
        axs[1, 1].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.show()

    else:
        # Scenario 3: Categorical x Categorical
        fig, axs = plt.subplots(2, 2, figsize=(16, 12))

        # Stacked Bar Chart
        crosstab = pd.crosstab(df[x], df[y])
        crosstab_perc = crosstab.div(crosstab.sum(1), axis=0)
        crosstab_perc.plot(kind='bar', stacked=True, ax=axs[0, 0])
        axs[0, 0].set_title('Stacked Bar Chart')
        axs[0, 0].tick_params(axis='x', rotation=45)

        # Add percentage labels on top of the bars for Stacked Bar Chart
        for container in axs[0, 0].containers:
            axs[0, 0].bar_label(container, labels=[f'{v * 100:.1f}%' for v in container.datavalues], label_type='center')

        # Grouped Bar Chart
        crosstab.plot(kind='bar', ax=axs[0, 1])
        axs[0, 1].set_title('Grouped Bar Chart')
        axs[0, 1].tick_params(axis='x', rotation=45)

        # Add values on top of the bars for Grouped Bar Chart
        for container in axs[0, 1].containers:
            axs[0, 1].bar_label(container)

        # Mosaic Plot
        mosaic(df, [x, y], ax=axs[1, 0])
        axs[1, 0].set_title('Mosaic Plot')
        axs[1, 0].tick_params(axis='x', rotation=45)

        # Count Plot
        sns.countplot(x=x, hue=y, data=df, ax=axs[1, 1])
        axs[1, 1].set_title('Count Plot')
        axs[1, 1].tick_params(axis='x', rotation=45)

        plt.tight_layout()
        plt.show()